In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
right = np.loadtxt("right_data.txt")
left = np.loadtxt("left_data.txt")
both_fists = np.loadtxt("both_fists_data.txt")
both_feet = np.loadtxt("both_feet_data.txt")

right_reshaped = np.reshape(right, (right.shape[0], right.shape[1] // 2, 2))
left_reshaped = np.reshape(left, (left.shape[0], left.shape[1] // 2, 2))
both_fists_reshaped = np.reshape(both_fists, (both_fists.shape[0], both_fists.shape[1] // 2, 2))
both_feet_reshaped = np.reshape(both_feet, (both_feet.shape[0], both_feet.shape[1] // 2, 2))

right_label = np.zeros((right.shape[0], 4))
right_label[:, 0] = 1
left_label = np.zeros((left.shape[0], 4))
left_label[:, 1] = 1
both_fists_label = np.zeros((both_fists.shape[0], 4))
both_fists_label[:, 2] = 1
both_feet_label = np.zeros((both_feet.shape[0], 4))
both_feet_label[:, 3] = 1

In [3]:
print(right_reshaped.shape)
print(left_reshaped.shape)
print(both_fists_reshaped.shape)
print(both_feet_reshaped.shape)

(2352, 640, 2)
(2396, 640, 2)
(2375, 640, 2)
(2372, 640, 2)


In [4]:
data = np.concatenate((right_reshaped, left_reshaped, both_fists_reshaped, both_feet_reshaped), axis = 0)
labels = np.concatenate((right_label, left_label, both_fists_label, both_feet_label), axis = 0)
print(data.shape)
print(labels.shape)
print(np.max(data))
print(np.min(data))

(9495, 640, 2)
(9495, 4)
0.000783
-0.000685


In [5]:
randomize = np.arange(0, data.shape[0], 1)
randomize = np.random.permutation(randomize)

data = data[randomize]
labels = labels[randomize]

print(data.shape)
print(labels.shape)

(9495, 640, 2)
(9495, 4)


In [6]:
training_x, training_y = data[0:8544], labels[0:8544]
testing_x, testing_y = data[8545:], labels[8545:]

In [7]:
normalizer = keras.layers.experimental.preprocessing.Normalization(axis=-1, mean=0., variance=1.)
normalizer.adapt(training_x)

In [8]:
inputs = keras.Input(shape=(640, 2, 1))

In [9]:
from tensorflow.keras import layers

x = layers.Conv2D(filters=25, kernel_size=[11,1], strides=[1,1], padding="valid")(inputs)
x = layers.LeakyReLU(alpha=0.01)(x)
x = layers.SpatialDropout2D(rate=0.5)(x)

x = layers.Conv2D(filters=25, kernel_size=[1,2], strides=[1,1], padding="valid")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.01)(x)

x = layers.MaxPooling2D(pool_size=[3,1], strides = [3,1], padding="valid")(x)

x = layers.Conv2D(filters=50, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.LeakyReLU(alpha=0.01)(x)
x = layers.SpatialDropout2D(rate=0.5)(x)

x = layers.MaxPooling2D(pool_size=[3,1], strides = [3,1], padding="valid")(x)

x = layers.Conv2D(filters=100, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.01)(x)
x = layers.SpatialDropout2D(rate=0.5)(x)

x = layers.MaxPooling2D(pool_size=[3,1], strides = [3,1], padding="valid")(x)

x = layers.Conv2D(filters=200, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.01)(x)

x = layers.MaxPooling2D(pool_size=[2,1], strides = [2,1], padding="valid")(x)

x = layers.Flatten()(x)

num_classes = 4
outputs = layers.Dense(num_classes, activation="softmax")(x)

In [10]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 640, 2, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 630, 2, 25)        300       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 630, 2, 25)        0         
_________________________________________________________________
spatial_dropout2d (SpatialDr (None, 630, 2, 25)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 630, 1, 25)        1275      
_________________________________________________________________
batch_normalization (BatchNo (None, 630, 1, 25)        100       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 630, 1, 25)        0     

In [18]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              loss=keras.losses.CategoricalCrossentropy(), metrics=keras.metrics.CategoricalAccuracy())

In [19]:
history = model.fit(training_x, training_y,
          batch_size=128, epochs=10)

Epoch 1/10
67/67 [==============================] - 11s 149ms/step - loss: 1.3862 - categorical_accuracy: 0.2484
Epoch 2/10
67/67 [==============================] - 10s 144ms/step - loss: 1.3863 - categorical_accuracy: 0.2508
Epoch 3/10
67/67 [==============================] - 10s 150ms/step - loss: 1.3861 - categorical_accuracy: 0.2546
Epoch 4/10
67/67 [==============================] - 11s 159ms/step - loss: 1.3863 - categorical_accuracy: 0.2471
Epoch 5/10
67/67 [==============================] - 11s 158ms/step - loss: 1.3862 - categorical_accuracy: 0.2554
Epoch 6/10
67/67 [==============================] - 11s 160ms/step - loss: 1.3861 - categorical_accuracy: 0.2540
Epoch 7/10
67/67 [==============================] - 10s 153ms/step - loss: 1.3858 - categorical_accuracy: 0.2592
Epoch 8/10
67/67 [==============================] - 11s 160ms/step - loss: 1.3862 - categorical_accuracy: 0.2525
Epoch 9/10
67/67 [==============================] - 10s 151ms/step - loss: 1.3861 - categorical_